<a href="https://colab.research.google.com/github/OlegV12/Rec-Sys/blob/Lesson_2/hw_webinar_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install implicit

     |████████████████████████████████| 1.1 MB 9.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for implicit: filename=implicit-0.4.8-cp37-cp37m-linux_x86_64.whl size=4606647 sha256=2e27a11ae46491a626f53530db3946ecaf0b1c9f570ae01c6587c2c7ee18f405
  Stored in directory: /root/.cache/pip/wheels/88/e6/34/25e73cccbaf1a961154bb562a5f86123b68fdbf40e306073d6
Successfully built implicit


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

ModuleNotFoundError: ignored

In [5]:
from google.colab import drive
drive.mount('/gdrive')



Mounted at /gdrive


In [6]:
data = pd.read_csv('/gdrive/MyDrive/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [7]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()

In [8]:
popularity['weight'] =popularity['sales_value'] * (1/popularity['sales_value'].sum())

In [9]:
popularity.sort_values('sales_value', ascending=False)

,item_id,sales_value,weight
56233,6534178,467993.62,0.062970
56193,6533889,42645.75,0.005738
29195,1029743,37981.91,0.005111
56228,6534166,31298.96,0.004211
35054,1082185,27291.02,0.003672
...,...,...,...
50764,2961522,0.00,0.000000
70059,10121805,0.00,0.000000
39334,1121502,0.00,0.000000
5918,821773,0.00,0.000000


In [10]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [11]:
def random_recommendation(items, n=5, p=None):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False, p=p)
    
    return recs.tolist()

In [12]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    recs = np.random.choice(items_weights['item_id'], size=n, replace=False, p=items_weights['weight'])

    
    return recs.tolist()

In [14]:
# your_code

weighted_random_recommendation(popularity)

[1055911, 7410168, 9527290, 995242, 865456]

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [3]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [29]:
result = pd.read_csv('/gdrive/MyDrive/predictions_basic.csv',)
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[553111, 990130, 14025228, 8091299, 855288]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[8155825, 1841061, 946817, 1541203, 8203466]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [30]:
columns = ['actual', 'random_recommendation', 'popular_recommendation',
       'itemitem', 'cosine', 'tfidf', 'own_purchases',]

In [31]:
for itm in columns:
  result[itm] = [[int(idx) for idx in list(filter(None, x.replace('\n', ' ').replace(',', '').strip('[').strip(']').strip('\n').strip(' ').split(' ')))] for x in result[itm]]

In [32]:
columns.remove('actual')

In [33]:
precision = result.copy()

In [34]:
for itm in columns:
  precision[itm] = result.apply(lambda x: precision_at_k(x[itm], x['actual']), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in long_scalars
  # This is added back by InteractiveShellApp.init_path()


In [35]:
precision = precision[['random_recommendation', 'popular_recommendation', 'itemitem', 'cosine',	'tfidf',	'own_purchases']].mean()

In [36]:
precision

random_recommendation     0.000392
popular_recommendation    0.155240
itemitem                  0.136925
cosine                    0.132909
tfidf                     0.138981
own_purchases             0.201917
dtype: float64

Лучшее качество показала рекомандация купленых товаров

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code